In [ ]:
import pandas as pd
import folium
from branca.colormap import LinearColormap
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Daten einlesen und vorverarbeiten
flights_routes_df = pd.read_csv(r"C:\Users\rober\OneDrive\Vorlesungen\Datenbasierte Fallstudien\Visualisierungen\Flights\Flights\flight_routes_summary.csv")


connections_df = flights_routes_df.groupby(['origin_city', 'destination_city', 'origin_airport_lat', 'origin_airport_lon', 'destination_airport_lat', 'destination_airport_lon', 'year', 'month'])['count(flight_id)'].sum().reset_index()
connections_df.columns = ['origin_city', 'destination_city', 'origin_airport_lat', 'origin_airport_lon', 'destination_airport_lat', 'destination_airport_lon', 'year', 'month', 'total_flights']
connections_orig_airports_df = connections_df.groupby(['origin_city', 'origin_airport_lat', 'origin_airport_lon', 'destination_city', 'destination_airport_lat', 'destination_airport_lon', 'year', 'month']).sum('total_flights').reset_index()
cached_connections_df = flights_routes_df.groupby(['origin_city', 'destination_city', 'origin_airport_lat', 'origin_airport_lon', 'destination_airport_lat', 'destination_airport_lon', 'year', 'month'])['count(flight_id)'].sum().reset_index()
cached_connections_df.columns = ['origin_city', 'destination_city', 'origin_airport_lat', 'origin_airport_lon', 'destination_airport_lat', 'destination_airport_lon', 'year', 'month', 'total_flights']
cached_connections_orig_airports_df = cached_connections_df.groupby(['origin_city', 'origin_airport_lat', 'origin_airport_lon', 'destination_city', 'destination_airport_lat', 'destination_airport_lon', 'year', 'month']).sum('total_flights').reset_index()

# MinMax-Scaler
max_bewegungen = connections_orig_airports_df['total_flights'].max()
min_bewegungen = connections_orig_airports_df['total_flights'].min()

# Funktion, um die Flugbewegungen zu skalieren
def scale_bewegungen(flugbewegungen):
    return (flugbewegungen - min_bewegungen) / (max_bewegungen - min_bewegungen)

# Daten aggregieren, um die Top 30 Flughäfen zu bestimmen
top_airports = connections_orig_airports_df.groupby(['origin_airport_lat', 'origin_airport_lon', 'origin_city']).agg({
    'total_flights': 'sum'
}).reset_index().nlargest(30, 'total_flights')

# Dash-App initialisieren
app = dash.Dash(__name__, external_stylesheets=['https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css', 'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css'])

# Styles definieren
styles = {
    'sidebar': {'position': 'fixed', 'top': '-250px', 'left': '12.5%', 'right': '25.5%', 'height': '252px', 'max-width': '95%', 'padding': '20px', 'background-color': '#f8f9fa', 'transition': 'top 0.3s ease-in-out', 'z-index': 1000, 'overflow-y': 'auto', 'box-shadow': '0 6px 8px rgba(0, 0, 0, 0.1)'},
    'sidebar-open': {'top': '0'},
    'sidebar-header': {'display': 'flex', 'justify-content': 'space-between', 'align-items': 'center', 'margin-bottom': '20px'},
    'sidebar-content': {'display': 'flex', 'justify-content': 'space-between'},
    'filter-group': {'flex': '1', 'padding': '0 3px', 'margin-bottom': '10px'},
    'icon-container': {'position': 'fixed', 'top': '0px', 'left': '5px', 'display': 'flex', 'align-items': 'center', 'background-color': 'white', 'padding': '13px', 'border-radius': '65%', 'cursor': 'pointer', 'z-index': 1001, 'color': 'orange', 'font-size': '40px'},
    'selected-year': {'margin-left': '10px', 'font-size': '18px', 'font-weight': 'bold'},
    'icon': {'transition': 'transform 0.3s ease-in-out'},
    'icon-open': {'transform': 'rotate(180deg)'},
    'content': {'margin-top': '50px', 'padding': '20px', 'transition': 'margin-top 0.3s ease-in-out'},
    'content-open': {'margin-top': '300px'},
    'container': {'display': 'grid', 'gap': '5px', 'align-items': 'stretch', 'height': '100vh', 'max-width': '94%', 'width': '92%', 'margin-right': '0px', 'padding': '0 2px'},
    'card': {'background-color': '#E7EFF9', 'transition': 'all 0.3s ease'},
    'card-cancellations': {'background-color': '#FEECEC', 'transition': 'all 0.3s ease'},
    'info-main-value': {'font-size': '1rem', 'transition': 'font-size 0.3s ease'},
    'graph-container': {
        'margin-bottom': '2px',
        'margin-left': '10px',
        'margin-right': '0px',
        'height': '450px',
        'justify-content': 'center',  
        'align-items': 'center' 
    },
    'card-body': {
        'display': 'flex',
        'flex-direction': 'column',
        'justify-content': 'space-around',
        'height': '510px'  # Erhöhe die Höhe der Karte für bessere Platzierung
    }
}

# App-Layout definieren
app.layout = html.Div([
    dcc.Store(id='sidebar-state', data=False),
    html.Div([
        html.Div([
            html.H3('Filter'),
        ], style=styles['sidebar-header']),
        html.Div([
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label': str(year), 'value': year} for year in sorted(connections_df['year'].unique())],
                value=sorted(connections_df['year'].unique())[-1],
                clearable=False,
                placeholder='Jahr auswählen',
                style={'margin-bottom': '20px'}
            ),
            dcc.Dropdown(
                id='origin-dropdown',
                options=[{'label': 'Alle', 'value': 'all'}] + [{'label': city, 'value': city} for city in top_airports['origin_city'].unique()],
                value='all',
                clearable=False,
                placeholder='Startflughafen',
                style={'margin-bottom': '20px'}
            ),
            dcc.Dropdown(
                id='destination-dropdown',
                options=[{'label': 'Alle', 'value': 'all'}] + [{'label': city, 'value': city} for city in top_airports['origin_city'].unique()],
                value='all',
                clearable=False,
                placeholder='Landeflughafen',
                style={'margin-bottom': '20px'}
            ),
        ], style=styles['filter-group']),
    ], id='sidebar', className='sidebar', style=styles['sidebar']),
    html.Div([
        html.I(id="toggle-sidebar", n_clicks=0, className='fas fa-chevron-down', style=styles['icon']),
    ], style=styles['icon-container']),
    html.Div([
        html.H3(''),
        dbc.Row([
            dbc.Col(
                dbc.Card(
                    dbc.CardBody([
                            html.Div([
                                html.Div(id='all-flights-year', style={'width': '49%', 'display': 'inline-block', 'text-align': 'right', 'padding-right': '10px'}),
                                html.Div("|", style={'display': 'inline-block'}),
                                html.Div(id='all-flights-total', style={'width': '49%', 'display': 'inline-block', 'text-align': 'left', 'padding-left': '10px'})
                                ], style={'width': '100%', 'text-align': 'left', 'weight': 'bold'}),
                        html.Div([dcc.Graph(id='origin-top-flights', config={'displayModeBar': False})], style=styles['graph-container'])
                        
                    ], style=styles['card-body']),
                    style={'width': '90%', 'margin-bottom': '10px'}  # Volle Breite und angepasster Margin
                ),
            ),
        ]),
        html.Div(html.Iframe(id='map-iframe', srcDoc=None, width='90%', height='800'), style={'height': '800px'})
    ], id='content', className='content', style=styles['content'])
], className='container', style=styles['container'])

# Callbacks für die Sidebar und Content-Bereich
@app.callback(
    [Output('sidebar', 'style'),
     Output('content', 'style'),
     Output('toggle-sidebar', 'className'),
     Output('sidebar-state', 'data')],
    [Input('toggle-sidebar', 'n_clicks')],
    [State('sidebar-state', 'data')]
)
def toggle_sidebar(n_clicks, sidebar_state):
    if n_clicks:
        if sidebar_state:
            return styles['sidebar'], styles['content'], 'fas fa-chevron-down', False
        else:
            return {**styles['sidebar'], **styles['sidebar-open']}, {**styles['content'], **styles['content-open']}, 'fas fa-chevron-up', True
    return styles['sidebar'], styles['content'], 'fas fa-chevron-down', False

# Hilfsfunktion für die Formatierung der Flugzahlen
def format_k_or_m(value):
    if value >= 1000000:
        return f"{value / 1000000:.1f}".replace('.', ',') + ' M'
    elif value >= 1000:
        return f"{value / 1000:.1f}".replace('.', ',') + ' K'
    return str(value)

# Callback für die Aktualisierung der Visualisierungen basierend auf der Auswahl
@app.callback(
    [
        Output('all-flights-year', 'children'),
        Output('all-flights-total', 'children'),
        Output('origin-top-flights', 'figure'),
        Output('map-iframe', 'srcDoc')
    ],
    [
        Input('origin-dropdown', 'value'),
        Input('destination-dropdown', 'value'),
        Input('year-dropdown', 'value')
    ]
)


def update_visualizations(selected_origin, selected_destination, selected_year):
    # Beginne mit den gesamten Daten und filtere dann, falls erforderlich
    filtered_data = connections_df[
        (connections_df['year'] == selected_year) &
        ((connections_df['origin_city'] == selected_origin) | (selected_origin == 'all')) &
        ((connections_df['destination_city'] == selected_destination) | (selected_destination == 'all'))
    ]
   
    # Aggregate die Daten neu, basierend auf der Filterung
    origin_flights_by_month = filtered_data.groupby(['month', 'origin_city'])['total_flights'].sum().reset_index()
    total_flights_per_city = filtered_data.groupby('origin_city', as_index=False)['total_flights'].sum()
    sorted_cities = total_flights_per_city.sort_values(by='total_flights', ascending=False)
    top_origin_cities = sorted_cities['origin_city'].head(10).tolist()
   
    # Berechne die Gesamtflüge
    total_flights = filtered_data['total_flights'].sum()
   
    cols = 5  # Anpassen an die Anzahl der darzustellenden Städte
    rows = int(len(top_origin_cities) / cols) + (len(top_origin_cities) % cols > 0)
    fig = make_subplots(rows=rows, cols=cols, subplot_titles=top_origin_cities)
    for index, city in enumerate(top_origin_cities, start=1):
        row = (index - 1) // cols + 1
        col = (index - 1) % cols + 1
        city_data = origin_flights_by_month[origin_flights_by_month['origin_city'] == city]
        fig.add_trace(
            go.Scatter(x=city_data['month'], y=city_data['total_flights'], mode='lines', name=city, line=dict(color='red')),
            row=row, col=col
        )
   
    # Setze die y-Achse auf die gleiche Skalierung für alle Diagramme
    max_flights = origin_flights_by_month['total_flights'].max()*1.1
    min_flights = origin_flights_by_month['total_flights'].min()
    fig.update_yaxes(range=[min_flights, max_flights])

    fig.update_layout(
      height=220 * rows,
        width=380 * cols,
        title_text='Flüge nach Top-10 Startflughäfen pro Monat',
        title_y=0.99,
        title_x=0,
        showlegend=False,
        paper_bgcolor='rgba(0,0,0,0)',   
        #plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=60, r=30, t=70, b=0)    
        )
    fig.update_xaxes(
        title_text='',
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext=['Jan', 'Feb', 'Mär', 'Apr', 'Mai', 'Jun', 'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez']
        )
    fig.update_yaxes(title_text='', row=1)

   
    # Karte aktualisieren
    m = folium.Map(location=[34.0522, -118.2437], zoom_start=5, tiles='CartoDB Positron')
   
    # Gruppiere die Linien nach Start- und Zielflughafen
    grouped_lines = filtered_data.groupby(['origin_airport_lat', 'origin_airport_lon', 'destination_airport_lat', 'destination_airport_lon'])['total_flights'].sum().reset_index()
   
    # Füge gruppierte Linien zur Karte hinzu
    for _, row in grouped_lines.iterrows():
        scaled_bewegungen = scale_bewegungen(row['total_flights'])
        opacity = scaled_bewegungen  
        
        line = folium.PolyLine(
            locations=[
                (row['origin_airport_lat'], row['origin_airport_lon']),
                (row['destination_airport_lat'], row['destination_airport_lon'])
            ],
            color=f'rgba(255, 0, 0, {opacity*0.08})',
            weight=scaled_bewegungen*0.9,
        )
        line.add_to(m)
   
    # Füge Marker für die 30 Städte mit den meisten Flugbewegungen hinzu
    for _, airport in top_airports.iterrows():
        folium.Marker(
            location=[airport['origin_airport_lat'], airport['origin_airport_lon']],
            popup=f"{airport['origin_city']}// Flights: {airport['total_flights']}",
            icon=folium.Icon(icon='plane', prefix='fa', color='gray')
        ).add_to(m)
   
    return (
        f"Jahr: {selected_year}",
        f"Total Flights: {format_k_or_m(total_flights)}",
        fig,
        m._repr_html_()
)

if __name__ == '__main__':
    app.run_server(debug=True)